# Simulating Priority-Based Vessel Operations with Manual Resource Management at a Shipping Terminal Using SimPy
#### Note: Manual Control Over Resource Release, Resource requests made without using 'with' satement

In [41]:
import simpy
import random
import heapq

#### Constants

In [42]:
AVG_ARRIVAL_TIME = 5 * 60  # Average arrival time in minutes (5 hours)
CONTAINER_COUNT = 150 # Provided number of containers in each vessel
CRANE_MOVE_TIME = 3  # Crane move time per container in minutes
TRUCK_ROUND_TRIP_TIME = 6  # Truck round trip time in minutes
SIMULATION_TIME = 24 * 60  # Simulation time in minutes (24 hours)

#### Time conversion function

In [43]:
def convert_minutes_to_hhmm(minutes):
    hours = int(minutes // 60)
    minutes = int(minutes % 60)
    return hours, minutes

#### Priority Request

In [44]:
# Priority Resource Request
class PriorityRequest:
    def __init__(self, priority, item):
        self.priority = priority
        self.item = item

    def __lt__(self, other):
        return self.priority < other.priority

#### Main function

In [45]:
# Functions
def vessel(env, name, priority, terminal):
    arrival_time = env.now
    arrival_hours, arrival_minutes = convert_minutes_to_hhmm(arrival_time)
    print(f"{name} (Priority {priority}) arrives at {arrival_hours} hours and {arrival_minutes} minutes")

    # Request berth with priority
    berth_req = PriorityRequest(priority, terminal.berths.request())
    heapq.heappush(terminal.berth_queue, berth_req)
    yield berth_req.item

    berthing_time = env.now
    berthing_hours, berthing_minutes = convert_minutes_to_hhmm(berthing_time)
    waiting_time = berthing_time - arrival_time
    waiting_hours, waiting_minutes = convert_minutes_to_hhmm(waiting_time)
    print(f"{name} berths at {berthing_hours} hours and {berthing_minutes} minutes after waiting for {waiting_hours} hours and {waiting_minutes} minutes")

    try:
        # Start unloading containers
        for i in range(CONTAINER_COUNT):
            # Request crane with priority
            crane_req = PriorityRequest(priority, terminal.cranes.request())
            heapq.heappush(terminal.crane_queue, crane_req)
            yield crane_req.item

            container_time = env.now
            container_hours, container_minutes = convert_minutes_to_hhmm(container_time)
            print(f"{name}: Crane starts unloading container {i+1} at {container_hours} hours and {container_minutes} minutes")

            # Move container
            yield env.timeout(CRANE_MOVE_TIME)

            # Place container on truck with priority
            truck_req = PriorityRequest(priority, terminal.trucks.request())
            heapq.heappush(terminal.truck_queue, truck_req)
            yield truck_req.item

            truck_time = env.now
            truck_hours, truck_minutes = convert_minutes_to_hhmm(truck_time)
            print(f"{name}: Crane places container {i+1} on truck at {truck_hours} hours and {truck_minutes} minutes")

            # Transport container to yard block
            print(f"Truck starts transporting container {i+1} to yard block at {truck_hours} hours and {truck_minutes} minutes")
            yield env.timeout(TRUCK_ROUND_TRIP_TIME)
            return_time = env.now
            return_hours, return_minutes = convert_minutes_to_hhmm(return_time)
            print(f"Truck returns to quay crane at {return_hours} hours and {return_minutes} minutes")

            # Release the truck
            terminal.trucks.release(truck_req.item)

            # Release the crane
            terminal.cranes.release(crane_req.item)

    finally:
        # Release berth
        terminal.berths.release(berth_req.item)
        
        # Vessel departs
        departure_time = env.now
        departure_hours, departure_minutes = convert_minutes_to_hhmm(departure_time)
        print(f"{name} departs at {departure_hours} hours and {departure_minutes} minutes")

#### Generator function

In [46]:
def vessel_arrival_generator(env, terminal):
    i = 0
    while True:
        # Yield a timeout that follows an exponential distribution
        yield env.timeout(random.expovariate(1 / AVG_ARRIVAL_TIME))
        priority = random.choice([1, 2])  # Randomly assign a priority (1 or 2)
        env.process(vessel(env, f"Vessel {i+1}", priority, terminal))
        i += 1

In [47]:
# Terminal Class with Priority Queues
class Terminal:
    def __init__(self, env):
        self.env = env
        self.berths = simpy.Resource(env, capacity=2)
        self.cranes = simpy.Resource(env, capacity=2)
        self.trucks = simpy.Resource(env, capacity=3)
        self.berth_queue = []
        self.crane_queue = []
        self.truck_queue = []

#### Simulation environment

In [48]:
# Simulation Setup
env = simpy.Environment()
terminal = Terminal(env)
env.process(vessel_arrival_generator(env, terminal))
env.run(until=SIMULATION_TIME)

Vessel 1 (Priority 2) arrives at 4 hours and 54 minutes
Vessel 1 berths at 4 hours and 54 minutes after waiting for 0 hours and 0 minutes
Vessel 1: Crane starts unloading container 1 at 4 hours and 54 minutes
Vessel 1: Crane places container 1 on truck at 4 hours and 57 minutes
Truck starts transporting container 1 to yard block at 4 hours and 57 minutes
Truck returns to quay crane at 5 hours and 3 minutes
Vessel 1: Crane starts unloading container 2 at 5 hours and 3 minutes
Vessel 1: Crane places container 2 on truck at 5 hours and 6 minutes
Truck starts transporting container 2 to yard block at 5 hours and 6 minutes
Truck returns to quay crane at 5 hours and 12 minutes
Vessel 1: Crane starts unloading container 3 at 5 hours and 12 minutes
Vessel 1: Crane places container 3 on truck at 5 hours and 15 minutes
Truck starts transporting container 3 to yard block at 5 hours and 15 minutes
Truck returns to quay crane at 5 hours and 21 minutes
Vessel 1: Crane starts unloading container 4 at